# PROJECT 6

# PIPELINE 2: SEARCH

# BONUS: 
Present: Command is executed via command line script

In [1]:
!pwd

/Users/ginodefalco/dsi/dsi-workspace/Projects/project_6


In [2]:
import os
os.chdir('./lib')

In [3]:
import database_module_updt as db
import encoding_module as enc
import wiki_module_copy as wiki
import pandas as pd

In [4]:
db.connect_to_postgres()

Connected to server joshuacook.me.


(<connection object at 0x1158bb180; dsn: 'user=gino password=xxxxxxxxxxx she wagon equator' host=joshuacook.me port=5432 dbname=project_6', closed: 0>,
 <cursor object at 0x1157df908; closed: 0>)

# Parse: Encode search as document vector


In [5]:
page_vectors = db.select_all_page_vectors()

Connected to server joshuacook.me.


In [6]:
indices = [tup[0] for tup in page_vectors]
vectors = [tup[1] for tup in page_vectors]

In [7]:
page_vectors_df = pd.DataFrame(vectors, index=indices)

In [8]:
page_vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
4341789,0.092992,-0.040985,0.006952,0.000243,0.001681,0.004999,0.003430,0.003523,-0.005129,-0.001911,...,-0.021003,-0.039528,0.027716,0.021033,-0.034768,-0.022294,0.031537,-0.009710,-0.010331,0.025944
48201744,0.021301,0.027559,-0.003917,-0.010877,0.013055,0.000934,0.000128,0.010317,0.001444,-0.009792,...,-0.055040,0.000110,0.057817,0.005156,0.058477,0.008807,0.043527,-0.011693,0.064738,0.051296
2514975,0.124983,0.153025,0.026264,-0.257191,-0.210183,0.074847,0.003862,-0.009666,-0.065705,-0.280972,...,-0.022703,0.015894,-0.002476,0.004992,-0.006653,-0.017394,-0.008525,0.017188,-0.001596,0.000120
35135520,0.191357,-0.039003,-0.006018,0.006389,-0.001106,-0.004748,0.015829,0.005612,0.011846,0.028248,...,0.000578,-0.004204,0.030916,0.010484,-0.012454,0.026050,0.038744,0.015332,-0.008053,0.009506
27303975,0.034292,0.039499,-0.006095,-0.015156,0.026112,-0.039749,-0.081440,-0.036546,0.005076,0.018881,...,0.030056,-0.026965,-0.002426,0.042282,0.024082,0.004498,-0.009517,-0.005315,-0.003784,-0.010202


In [9]:
page_vectors_df.shape

(2270, 500)

In [10]:
page_vectors_df[1].as_matrix()

array([-0.04098514,  0.02755917,  0.15302509, ...,  0.24935387,
        0.2834365 ,  0.08240345])

In [11]:
from sklearn.externals import joblib
transformer = joblib.load('/Users/ginodefalco/dsi/dsi-workspace/Projects/project_6/data/vectorizer(1).pkl')

In [12]:
search_term = 'steve jobs'

In [13]:
test_vec = enc.get_searchterm_vector(transformer, search_term)

In [14]:
test_vec.values()[0][0].shape

(500,)

In [15]:
test_vec = test_vec.values()[0][0]

In [16]:
test_vec

array([  3.52379698e-03,   1.55156385e-03,  -2.22391882e-03,
         1.48923200e-04,  -1.56076045e-03,   3.55996343e-04,
         3.66544673e-05,  -5.92638807e-04,  -9.80722891e-04,
         4.88593667e-04,  -1.92905728e-03,  -4.04103430e-04,
        -4.09921068e-04,   1.40910127e-03,  -1.03500250e-03,
         2.42885848e-03,   1.18846424e-04,  -2.61224733e-03,
         4.18605032e-04,  -3.25036946e-03,  -7.41455414e-04,
         2.96052564e-03,   1.55751352e-03,   8.22861752e-04,
        -5.76304049e-04,  -1.78838303e-03,   3.61818629e-03,
         1.37083318e-03,  -6.58150609e-04,   2.24037865e-04,
        -7.92612114e-04,  -1.44483477e-03,  -2.19441767e-03,
        -3.53477853e-03,  -2.85772562e-03,  -8.87745579e-04,
         1.43593606e-03,  -9.05940353e-04,  -5.93635142e-04,
         3.39636792e-03,  -7.64174455e-04,  -8.88899052e-04,
        -3.36521482e-04,   8.57806795e-04,  -4.00816816e-04,
         1.56177825e-03,   3.89576296e-03,  -1.73228543e-03,
        -3.04187335e-03,

# Model: Find five most similar documents based on document vectors

In [17]:
# INDEX ON PAGE ID
# Cosine similarity is one way to do this
# The other approach is nearest neighbors (not a classifier)
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [18]:
this_NN = NearestNeighbors()

In [19]:
this_NN.fit(page_vectors_df)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=5, p=2, radius=1.0)

In [20]:
# first array are distances, second five are the indices
tnn = this_NN.kneighbors(test_vec)
tnn

/Users/ginodefalco/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


(array([[ 0.25545628,  0.3564603 ,  0.37011145,  0.37979901,  0.38626868]]),
 array([[1172,  247,  795,  483,  566]]))

In [21]:
indices = tnn[1][0]
indices

array([1172,  247,  795,  483,  566])

In [22]:
pv = page_vectors_df.index[indices]
pv

Int64Index([8181732, 18007329, 2904185, 13846997, 38485157], dtype='int64')

# Mine: Retrieve text for the five most similar documents

In [23]:
db.select_pages(pv)

Connected to server joshuacook.me.


[(2904185,
  'Simit (Turkish: simit, Arabic: \xd8\xb3\xd9\x85\xd9\x8a\xd8\xb7\xe2\x80\x8e, translit. sam\xc4\xabt\xcb\xa4\xe2\x80\x8e, Persian: \xd8\xb3\xdb\x8c\xd9\x85\xdb\x8c\xd8\xaa\xe2\x80\x8e, translit',
  'Simit (Turkish: simit, Arabic: \xd8\xb3\xd9\x85\xd9\x8a\xd8\xb7\xe2\x80\x8e, translit. sam\xc4\xabt\xcb\xa4\xe2\x80\x8e, Persian: \xd8\xb3\xdb\x8c\xd9\x85\xdb\x8c\xd8\xaa\xe2\x80\x8e, translit. s\xc4\xabm\xc4\xabt\xcb\xa4\xe2\x80\x8e, Bulgarian: \xd1\x81\xd0\xb8\xd0\xbc\xd0\xb8\xd1\x82  simit ), gevrek (Turkish: gevrek, Bulgarian: \xd0\xb3\xd0\xb5\xd0\xb2\xd1\x80\xd0\xb5\xd0\xba, Macedonian: \xd1\x93\xd0\xb5\xd0\xb2\xd1\x80\xd0\xb5\xd0\xba, Serbian: \xd1\x92\xd0\xb5\xd0\xb2\xd1\x80\xd0\xb5\xd0\xba), bokegh (Armenian: \xd5\xa2\xd5\xb8\xd5\xaf\xd5\xa5\xd5\xb2), or koulouri (Greek: \xce\xba\xce\xbf\xcf\x85\xce\xbb\xce\xbf\xcf\x8d\xcf\x81\xce\xb9) is a circular bread, typically encrusted with sesame seeds or, less commonly, poppy, flax or sunflower seeds, found across the cuisines 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
cosine_similarity(search_vector_doc.values()[0],search_vector2.values()[0])[0][0] #Kobe Bryant

# Mine: Select document vectors for all pages from database

In [24]:
vecdict = {}
for page_id, vec in page_vectors:
    vecdict[page_id] = vec
    

In [25]:
vec_df = pd.DataFrame.from_dict(vecdict, orient='index')
vec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
17645570,0.039274,0.055023,-0.014005,-0.016520,0.022691,-0.018724,0.002756,0.005556,-0.005737,-0.009030,...,-0.010056,-0.014171,-0.013905,0.003224,0.005246,0.004800,0.032152,0.011491,0.022782,0.003275
37019651,0.101253,0.161288,-0.224323,0.112926,-0.060645,0.066798,0.023622,0.008715,-0.017338,0.008191,...,-0.001198,-0.016804,-0.015773,-0.009290,-0.000526,0.004315,-0.003473,0.003300,-0.009625,-0.005633
9109512,0.039232,0.039217,0.008973,-0.047728,-0.015612,0.017161,0.000355,-0.002134,-0.002816,-0.015128,...,0.031067,0.011480,-0.021530,-0.004341,0.004190,0.044702,0.011483,-0.013307,-0.031249,-0.063539
45260809,0.186930,-0.060678,-0.004299,0.012090,0.001639,-0.007242,0.005464,-0.005203,0.001449,0.013998,...,0.019338,-0.001007,0.019923,0.030012,0.018544,-0.058307,-0.000886,-0.040026,-0.012019,0.010476
6565890,0.067058,0.092235,-0.103929,0.040591,-0.002851,0.008336,-0.013674,-0.000052,0.021902,-0.000659,...,0.006890,0.005918,-0.003518,-0.013995,-0.002671,0.020586,0.001678,-0.004499,0.003813,-0.016608
